In [14]:
import os
from dotenv import load_dotenv

from langchain_openai import ChatOpenAI
from langchain.agents import load_tools, initialize_agent
from langchain.agents import AgentExecutor, create_react_agent, create_openai_tools_agent
from langchain.tools import AIPluginTool

from langchain import hub

In [15]:
# Load environment variables from .env file
load_dotenv()

# Access the environment variables
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

In [16]:
# Setup environment variables
URL = "https://blockchatstatic.blob.core.windows.net/api-configuration"

llm = ChatOpenAI(temperature=0)

# AI Agent does best when it only has one available tool
# to engage with URLs
tools = load_tools(["requests_post"], allow_dangerous_tools=True)

# AIPluginTool only fetches and returns the openapi.yaml linked to in /.well-known/ai-plugin.json
# This may need some more work to avoid blowing up LLM context window
tool = AIPluginTool.from_plugin_url(URL + "/.well-known/ai-plugin.json")
tools += [tool]

In [17]:
from langchain_core.prompts import PromptTemplate

template = '''Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought: {agent_scratchpad}'''

# prompt = PromptTemplate.from_template(template)
prompt = hub.pull("blockchain-researcher-prompt")

In [18]:
# Setup an agent to answer the question without further human feedback
# agent = create_react_agent(
#     llm, tools, prompt)
agent = create_openai_tools_agent(
    llm, tools, prompt)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True)

agent_executor.invoke({"input": "How many lamports does 8fbqVvpK3Dj7fdP2c8JJhtD7Zy3n9qtwAeGfbkgPu625 have?"})



> Entering new AgentExecutor chain...


KeyError: "Input to ChatPromptTemplate is missing variables {'messages'}.  Expected: ['agent_scratchpad', 'messages'] Received: ['input', 'intermediate_steps', 'agent_scratchpad']"